# Process single cell morphology features for CellProfiler readouts - Plate 2

## Import Libraries

In [1]:
import pathlib
import pandas as pd

from pycytominer import normalize, feature_select
from pycytominer.cyto_utils import cells, output

## Set up paths to CellProfiler directory and outputs

In [2]:
# Set file and directory constants
cp_dir = "../../../CellProfiler_pipelines"
output_dir = "../../data/Plate2/CellProfiler"

## Set paths to sqlite files

In [3]:
# Set name and path of .sqlite file and path to metadata
sql_file_pbcp = "NF1_data_pybasic_cp_plate2.sqlite"
single_cell_file_pbcp = f"sqlite:///{cp_dir}/Analysis_Output/Plate2_Output/{sql_file_pbcp}"
platemap_file = f"{cp_dir}/Metadata/platemap_NF1_CP_Plate2.csv"

# Set path with name for outputted data
sc_output_file_pbcp = pathlib.Path(f"{output_dir}/nf1_sc_pybasic_cp_plate2.csv.gz")
sc_norm_output_file_pbcp = pathlib.Path(f"{output_dir}/nf1_sc_norm_pybasic_cp_plate2.csv.gz")
sc_norm_fs_output_file_pbcp = pathlib.Path(f"{output_dir}/nf1_sc_norm_fs_pybasic_cp_plate2.csv.gz")

## Set up names for linking columns between tables in the database file

In [4]:
# Define custom linking columns between compartments
linking_cols = {
    "Per_Cytoplasm": {
        "Per_Cells": "Cytoplasm_Parent_Cells",
        "Per_Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Per_Cells": {"Per_Cytoplasm": "Cells_Number_Object_Number"},
    "Per_Nuclei": {"Per_Cytoplasm": "Nuclei_Number_Object_Number"},
}

## Load and view platemap file

In [5]:
# Load platemap file
platemap_df = pd.read_csv(platemap_file)
platemap_df.head()

,WellRow,WellCol,well_position,gene_name,genotype
0,A,1,A1,NF1,WT
1,A,6,A6,NF1,WT
2,A,7,A7,NF1,Null
3,A,12,A12,NF1,Null
4,B,1,B1,NF1,WT


## Set up `SingleCells` class from Pycytominer

In [6]:
# Instantiate SingleCells class
sc_pbcp = cells.SingleCells(
    sql_file=single_cell_file_pbcp,
    compartments=["Per_Cells", "Per_Cytoplasm", "Per_Nuclei"],
    compartment_linking_cols=linking_cols,
    image_table_name="Per_Image",
    strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
    merge_cols=["ImageNumber"],
    image_cols="ImageNumber",
    load_image_data=True
)

/home/jenna/anaconda3/envs/4.process-nf1-features/lib/python3.8/site-packages/pycytominer/cyto_utils/util.py:61: UserWarning: Non-canonical compartment detected: per_cells, per_cytoplasm, per_nuclei
  warnings.warn(warn_str)


## Merge single cells 

In [7]:
# Merge single cells across compartments
anno_kwargs = {"join_on": ["Metadata_well_position", "Image_Metadata_Well"]}

sc_df_pbcp = sc_pbcp.merge_single_cells(
    platemap=platemap_df,
    **anno_kwargs,
)

# Save level 2 data as a csv
output(sc_df_pbcp, sc_output_file_pbcp)

print(sc_df_pbcp.shape)
sc_df_pbcp.head()

(1693, 1209)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,A,1,NF1,WT,8,001,A1,1,5,1,...,550.888086,543.721759,332.513785,323.209698,323.720279,306.419718,45.724648,46.973859,46.312279,45.530083
1,A,1,NF1,WT,8,001,A1,2,7,2,...,383.161349,389.77554,84.295823,82.660081,86.747543,84.583084,25.902096,24.730391,24.62109,24.964434
2,A,1,NF1,WT,8,001,A1,3,8,3,...,270.128389,266.210486,233.402864,236.523247,227.538804,228.017641,44.75104,42.546228,44.411687,45.54969
3,A,1,NF1,WT,8,001,A1,4,9,4,...,124.575863,122.978334,25.33525,24.338147,24.147088,24.765955,7.564474,7.384528,7.418935,7.477577
4,A,1,NF1,WT,8,001,A1,5,11,5,...,263.38522,267.494535,177.41137,183.136849,182.217278,174.622946,60.570681,60.456594,60.752328,61.085318


## Normalize Data

In [8]:
# Normalize single cell data and write to file
normalize_sc_df_pbcp = normalize(
    sc_df_pbcp,
    method="standardize"
)

output(normalize_sc_df_pbcp, sc_norm_output_file_pbcp)

print(normalize_sc_df_pbcp.shape)
normalize_sc_df_pbcp.head()

(1693, 1209)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,A,1,NF1,WT,8,001,A1,1,5,1,...,-0.159663,-0.161597,0.190198,0.179504,0.169331,0.135590,-0.319016,-0.308200,-0.318515,-0.315281
1,A,1,NF1,WT,8,001,A1,2,7,2,...,-0.354217,-0.342818,-0.455181,-0.456125,-0.450272,-0.452081,-0.413685,-0.415198,-0.423041,-0.414288
2,A,1,NF1,WT,8,001,A1,3,8,3,...,-0.485329,-0.488276,-0.067495,-0.049556,-0.082151,-0.072106,-0.323666,-0.329498,-0.327673,-0.315187
3,A,1,NF1,WT,8,001,A1,4,9,4,...,-0.654162,-0.656885,-0.608481,-0.610235,-0.613951,-0.610543,-0.501262,-0.498636,-0.505936,-0.498472
4,A,1,NF1,WT,8,001,A1,5,11,5,...,-0.493151,-0.486764,-0.213076,-0.190625,-0.200651,-0.213555,-0.248115,-0.243344,-0.248930,-0.240396


## Feature Selection

In [9]:
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

feature_select_norm_sc_df_pbcp = feature_select(
    normalize_sc_df_pbcp,
    operation=feature_select_ops
)

output(feature_select_norm_sc_df_pbcp, sc_norm_fs_output_file_pbcp)

print(feature_select_norm_sc_df_pbcp.shape)
feature_select_norm_sc_df_pbcp.head()

(1693, 411)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_InfoMeas2_RFP_3_00_256,Nuclei_Texture_InfoMeas2_RFP_3_01_256,Nuclei_Texture_InfoMeas2_RFP_3_02_256,Nuclei_Texture_InfoMeas2_RFP_3_03_256,Nuclei_Texture_InverseDifferenceMoment_DAPI_3_02_256,Nuclei_Texture_InverseDifferenceMoment_GFP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_02_256,Nuclei_Texture_SumEntropy_DAPI_3_02_256,Nuclei_Texture_SumVariance_GFP_3_03_256,Nuclei_Texture_SumVariance_RFP_3_03_256
0,A,1,NF1,WT,8,001,A1,1,5,1,...,-0.020919,-0.524805,-0.477457,-0.264731,-0.305696,-0.933399,-0.023194,0.675776,0.056061,-0.295643
1,A,1,NF1,WT,8,001,A1,2,7,2,...,-0.507459,-1.033030,-1.277980,-0.606529,-0.394412,0.582421,0.255594,0.306081,-0.440933,-0.371191
2,A,1,NF1,WT,8,001,A1,3,8,3,...,0.916494,0.808525,1.055298,1.264028,0.016830,-0.750907,2.118217,-0.143738,-0.068483,-0.243508
3,A,1,NF1,WT,8,001,A1,4,9,4,...,-2.057541,-0.007570,-0.706232,-2.157045,0.103811,1.589699,3.042978,-1.110407,-0.579076,-0.444132
4,A,1,NF1,WT,8,001,A1,5,11,5,...,-0.088065,-0.520026,-0.347130,0.429166,0.005212,-0.353840,-0.614683,-0.410065,-0.226422,-0.199859


---

### Visualize basic count statistics

In [10]:
sc_df_pbcp.Metadata_genotype.value_counts()

Null    1000
WT       693
Name: Metadata_genotype, dtype: int64

In [11]:
pd.crosstab(sc_df_pbcp.Metadata_genotype, sc_df_pbcp.Metadata_Well)

Metadata_Well,A1,A12,A6,A7,B1,B12,B6,B7,C1,C12,...,F6,F7,G1,G12,G6,G7,H1,H12,H6,H7
Metadata_genotype,,,,,,,,,,,,,,,,,,,,,
Null,0,44,0,53,0,48,0,59,0,64,...,0,77,0,54,0,72,0,53,0,59
WT,48,0,51,0,49,0,48,0,55,0,...,44,0,55,0,33,0,26,0,29,0
